Loading Dataset




-Train and test data splitting


In [7]:
import pandas as pd

#creating train dataset using random rows from the combined dataset
df = pd.read_csv(r'G:\Diksha\7 sem\Final yearproject\NLP-text-doc-classification\Dataset\combined_final_dataset.csv')
df = df.sample(n=1367, random_state=1)
#save the csv to dataset folder
df.to_csv('combined_final_dataset_train.csv', index=False) #saving the training dataset


In [9]:
#creating test dataset using random rows from the combined dataset
df = pd.read_csv(r'G:\Diksha\7 sem\Final yearproject\NLP-text-doc-classification\Dataset\combined_final_dataset.csv')
df = df.sample(n=100, random_state=1)
#save in datset folder

df.to_csv('combined_final_dataset_test.csv', index=False) #saving the test csv


In [10]:
df_train = pd.read_csv('combined_final_dataset_train.csv')
df_test = pd.read_csv('combined_final_dataset_test.csv')

In [ ]:
#drop columns Title. category and date
df_test = df_test.drop(['Title','category','date'],axis = 1)

In [ ]:
df_train = df_train.drop(['Title','category','date'],axis = 1)

In [ ]:
# Assuming df_train and df_test are your DataFrames
encoded_dict = {
    'News': 0,
    'Research Paper': 1,
    'Code': 2,
    'Medical': 3,
    'Legal': 4,
    'Financial documents': 5
}

df_train['main_category'] = df_train['main_category'].map(encoded_dict)
df_test['main_category'] = df_test['main_category'].map(encoded_dict)


In [ ]:
from tensorflow.keras.utils import to_categorical
from keras.utils import to_categorical

In [ ]:
y_train = to_categorical(df_train.main_category)
y_test = to_categorical(df_test.main_category)


In [ ]:
import transformers
from transformers import AutoTokenizer,TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

In [ ]:
#datatypes of description
df_train.description.dtype
#convert desription column to string
df_train['description'] = df_train['description'].astype(str)
df_test['description'] = df_test['description'].astype(str)


In [ ]:
# Tokenize the input (takes some time) 
# here tokenizer using from bert-base-cased
x_train = tokenizer(
    text=df_train.description.tolist(),
    add_special_tokens=True,
    max_length=300,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
x_test = tokenizer(
    text=df_test.description.tolist(),
    add_special_tokens=True,
    max_length=300,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [ ]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

In [ ]:
max_len = 300
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)
y = Dense(6,activation = 'sigmoid')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [ ]:
optimizer = Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    epsilon=1e-08,
    clipnorm=1.0)
# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [ ]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, y_test
    ),
  epochs=1,
    batch_size=36
)

In [ ]:
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
predicted_raw[0]

In [ ]:
import numpy as np
y_predicted = np.argmax(predicted_raw, axis = 1)
y_true = df_test.main_category

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_predicted))

In [ ]:
texts = 'Arrhythmia or irregular heartbeat is a condition in which the heart  is unable to pump blood to the body efficiently. Symptoms of arrhythmia include: Fluttering in the chest Pounding heartbeat'
x_val = tokenizer(
    text=texts,
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True) 
validation = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100
for key , value in zip(encoded_dict.keys(),validation[0]):
    print(key,value)